__COMP9727 PROJECT:A STEAM GAME RECOMMENDER SYSTEM__


# Introduction
This model is a recommender system for Steam games. It is based on the dataset from Steam, which is a digital distribution platform for video games. The dataset contains 316 games and 21 million users. The recommender system is based on collaborative filtering and content based model. The system integrates the recommender module and a user interaction module.

__PIPELINE__:
1. Data preprocessing, Analysis and Visualization
2. Extracting features from the game dataset, build feature vectors for games. The item vector is a hierarchical structure. First, for the description, use text vectorization technology (such as TF-IDF or Word2Vec) to process the description text of the game to generate a dense numerical vector for each game. For Tags, we still use TF-IDF. But since the game tag describes the game more significantly, we manually increase the TF-IDF weight of the tag.

__Inporting Libraries__

In [218]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

__Data Preprocessing__

In [219]:
file_path_reviews = "steam_reviews.csv"
file_path_desc = "desc.csv"
file_path_tags = "tags.csv"

df_reviews = pd.read_csv(file_path_reviews)
df_desc = pd.read_csv(file_path_desc)
df_tags = pd.read_csv(file_path_tags)

__Clean the data__

In [220]:
import ast
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
import re
def tags_preprocess(tag):
    tag = ast.literal_eval(tag)
    tag = [x.lower() for x in tag]
    tag = [x.replace(' ', '_') for x in tag]
    tag = [x.replace('-', '_') for x in tag]
    tag = ' '.join(tag)
    return tag*4 # increase the weight of tags
def desc_preprocess(desc):
    desc = '' if pd.isna(desc) else str(desc)
    desc = desc.lower()
    desc = re.sub(r'[^\w\s]', '', desc)
    words = word_tokenize(desc)
    words = [word for word in words if word not in stopwords.words('english')]
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    text = ' '.join(words)
    return text

In [221]:
df_tags['tags'] = df_tags['tags'].apply(lambda x: tags_preprocess(x))
df_desc['short_description'] = df_desc['short_description'].apply(lambda x: desc_preprocess(x))



In [222]:
print(df_reviews)
print(df_desc)
print(df_tags)


          Unnamed: 0  app_id                  app_name  review_id  language  \
0                  0  292030  The Witcher 3: Wild Hunt   85185598  schinese   
1                  1  292030  The Witcher 3: Wild Hunt   85185250  schinese   
2                  2  292030  The Witcher 3: Wild Hunt   85185111  schinese   
3                  3  292030  The Witcher 3: Wild Hunt   85184605   english   
4                  4  292030  The Witcher 3: Wild Hunt   85184287  schinese   
...              ...     ...                       ...        ...       ...   
21747366    21747371  546560           Half-Life: Alyx   65645125   english   
21747367    21747372  546560           Half-Life: Alyx   65645115   english   
21747368    21747373  546560           Half-Life: Alyx   65645100   english   
21747369    21747374  546560           Half-Life: Alyx   65645066   english   
21747370    21747375  546560           Half-Life: Alyx   65644930  schinese   

          timestamp_created  timestamp_updated  rec

__Extracting Features from the description dataset and the tags dataset__

In [223]:
import ast
tfidf_vectorizer_tags = TfidfVectorizer()
tfidf_vectorizer_desc = TfidfVectorizer()
tfidf_matrix_tags = tfidf_vectorizer_tags.fit_transform(df_tags['tags'])
tfidf_matrix_desc = tfidf_vectorizer_desc.fit_transform(df_desc['short_description'])
print(tfidf_matrix_desc)
print(tfidf_matrix_tags)

In [224]:
from scipy.sparse import hstack
combined_tfidf_matrix = hstack((tfidf_matrix_desc, tfidf_matrix_tags))
print('Combined TF-IDF Matrix Shape:', combined_tfidf_matrix.shape)

Combined TF-IDF Matrix Shape: (315, 2974)


Create user class. The user class takes user model as input. A user could be an existed user in our reviews data set either a new user, needing cold start which will be implemented later.

In [225]:
class User:
    def __int__(self, user_mod, user_id):
        self.user_mod = user_mod
        self.user_id = user_id
        self.profile = [] # this array contains game ids.
        self.feature_arr = None # feature array(TF-IDF) depends on the profile array.
    def hot_start(self):
        pass
    def cold_start(self):
        pass
    def update_profile(self, app_id):
        self.profile.append(app_id)
    def generate_new_profile_vector(self):
        new_feature_arr = TfidfVectorizer()
    def like_game(self,app_id):
        self.update_profile(app_id)





